## RNA Energy Dot Plot Intro and Replicate for Folding Software

- RNA sequences fold into secondayr and tertiary -> driven by nucleotide base pair affinity
    - G like C, A likes U
- Each nucleotide binds with one partner at a time, long stretches where they have binding partner are more favourable than shorter, closer binding partners in terms of the sequence
    - this is why we don't get all small loops and hairpins across an elongated structure
- competition across base pairs creates folding based on available energy and the lowest-energy conformation state 
    - in some cases, the structure is optimized to fold such as certain codons align with other codons and typically form complementary strains that are just long, double-stranded RNA folds
        - however, arrangements are only probabalistic, to temperature and pH induces different arrangement, and can transition between different optimal structures in available structures
    - *All energy dot plots start at 5' end of the sequence

[Diagram](/Users/devpatelio/Desktop/Saturday.jpeg)

1. Start with diagonal and first point -> align the point along the x-axis and see whether it aligns with a base-pair
2. Assuming if there is no highlighted energy state at that intersection, it means that there are no base-pairs available
3. Repeating this process, if there is a highlighted energy state at the intersection, it means that the base pairs binds between the base-pair at x-coordinate and the base-pair at y-coordinate

![Diagram](/Users/devpatelio/Desktop/peen.png)

4. For loops, notice that when there are no highlighted states, it means that the base-pair at the x-coordinate has no binding partners so they branch away and become independent [often yields loop]
5. Once you reach a brief stretch, it can also potentially become a smaller stem branching off from the main loop 


Reference RNA secondary Structure Types [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6009582/)


### Supplemental: RNA Secondary Structure
- beginning of sequence = 5', end = 3'
1) Stem: more than one base pair appears in the form of a group of contiguous base pairs -> resulting structure motif described as a stem
    - for RNA, stem motif appears as flat but can always twist and turn at the tertiary level -> 
![](/Users/devpatelio/Downloads/ss1.png)

2) Hairpin: two complementary sequences joined by non-pairing bases in a loop [no bonds]
    - "AAAAACCCCUUUUU" -> bracket notation of this sequence indicates that CCCC will not bond due to lack of complementarity, and therefore forms hairpin loop
    - 2 stems on both sides -> 5'((((....))))3'
![](/Users/devpatelio/Downloads/hairpin.png)

3) Internal: appears between two stems with unpaired bases in between forming bulges on both sides -> mostly symmetrical internal loop where number of bases on each side are equal
    - n1, n2 represent both sides of the loop at 5' and 3', and there are 3 possibilities
    a) n1 > 0 and n2 = 0 [i=i'+1]
    b) n1 = 0 and n2 > 0 [j=j'+1]
    c) n1 > 0 and n2 > 0 [i, j, i', j' }- all fit in range]
![](/Users/devpatelio/Downloads/internal-bulge.png)

4) Multibranch: consist of several stem-loop type structures and very common
- while most other loops are detectable easier from traversing a RNA sequence from both ends and then inward, multiloops are more complex
- instead, you start from the center and expand outwards, and calculate energy of said structure such that you minimize total energy [nussinov approach]
![](/Users/devpatelio/Downloads/multi.png)




In [4]:
#Intro to ViennaFold

'''
First order of business: Learn about the concepts and algorithms
Secondary order of business: Write substack post on secondary structure types and data formatting
Third order of business: Go back to explore.ipynb and finish up BOoB
'''


import RNA

##MFE prediction from multiple sequence alignment
sequences = [
    "CUGCCUCACAACGUUUGUGCCUCAGUUACCCGUAGAUGUAGUGAGGGU",
    "CUGCCUCACAACAUUUGUGCCUCAGUUACUCAUAGAUGUAGUGAGGGU",
    "---CUCGACACCACU---GCCUCGGUUACCCAUCGGUGCAGUGCGGGU"]

cons = RNA.consensus(sequences) ###overlap sequences
(ss, mfe) = RNA.alifold(sequences) ###secondary structure, minimum free energy (MFE)
print(f"{cons}\n{ss}, {mfe}")


##Editing Default Model
seq = 'GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA'
md = RNA.md() #class for creating custom model
md.temperature = 20.0 #set default temperature
md.dangles = 1 #dangles are nucleotides with no pairs at start or ends of molecule

#fold compound refers model
fc = RNA.fold_compound(seq, md)
(ss, mfe) = fc.mfe() #predict MFE and secondary structure

## Turner's Rule and Stability
- how do you determine duplex stability -> created certain parameters that allow you to sum up the relative stability of each nearest neigbour pair in a duplex
- [Diagram](https://www.notion.so/devpatelio/RNA-Folding-and-Deep-Learning-46968b47a9bc4b8b9517cf6ee1cbeab5#1fe8fb1d12fb4204952e6a920536a1a3)
- these are all thermodynamic parameters where you can look at all possible 2-nucleotide pairs and observe their relative stability
- dictated by stacking of neighbouring base-pairs
    - for a given set of neighbours, you can quickly predict the free energy of stabilization
- ![](https://cdn.numerade.com/ask_images/f4c2dd04f582430c87efeb047302d372.jpg)

## Core Functionalities for Concepts and Algorithms in ViennaFold [RNAlib]
* This is adopted from the RNAlib library and acts as a brain-dump for documentation and all unknown concepts + anything else I shall add -> resources in directory

### Distance Measures
- measures dissimilarity between secondary structures that have equal length -> how many changes in base-pairs have to be made to get to the conformation of the second molecule
- useful for comparing structures on the same sequence regardless of codon sequence
    ``` int bp_distance(const char *str1, const char *str2) ```
- benefit of this is that you can calculate bp_distance across a wide matrix of possible mRNA conformations such that you can find distances across multiple kinds of mRNA or amino acid residues
- ```for two mRNA molecules i&j, the ij element of the matrix is 1 if i and j are closer than threshold x``` -> index resuls as an inequality with respect to filtering
- this creates a reduced representation of the coordinates and features of the different mRNA molecules without having to individually encode them
- this helps identify interactions between different kinds of chain-chain bonding and specific mRNA secondary structures (pseudoknots, hairpin loops, junctions, etc.) and we can index + sort this based on their fitness
- finding patterns on the similarity graphs can help in classifying specific structural conformations if ordered by distance [traversing low energy state]
- distances can be calculated between structures as edit distances between tree representations of the possible sequence [traverse branches and find difference in traversla length]
    - *reference to sequence alignment where you can look at all possible paths in the graph but need to satisfy specific conditions [each edit operation has a specific cost, and so the minimum edit distance is the minimum sum of costs along an edit path such that mRNA molecule ``` i ``` matches ``` j ```]
        - operations = insertion, deletion, and replacement of nodes -> ViennaRNA assignes insertion and deletions for single unpaired base = 1, for base pair = 2
- [reference](https://sci-hub.mksa.top/10.1093/bioinformatics/6.4.309)

### Free Energy of SS
- secondary structure free energy is determined by decomposing each individual loop to evaluate stability in terms of energy
    - each base pair in SS closes a loop, so by looking at each individual loop, ViennaRNA can easily determine the energy states by summing up all total energy states
1. Hairpin Loops
2. Interior Loops
3. Multibranch Loops
4. Exterior Loops

![poop](https://www.tbi.univie.ac.at/RNA/ViennaRNA/doc/html/loop_types.png)

- almost all secondary structures can be decomposed into its loops -> each loop can be scored and the free energy is simple sum -> ViennaRNA evaluates as follows: 
![](https://www.tbi.univie.ac.at/RNA/ViennaRNA/doc/html/loop_decomposition.png)
- Free Energy calculated with -> Boltzmann weight $q = exp(-\beta E)$ of the free energy $E$ (with $\beta = \frac{1}{RT}$, gas constant $R$ and temperature $T$)
    - Blotzmann is constant that relates the average relative kinetic energy of particles in a gas with thermodynamic temperature of the gas
- evaluating translation factors for mRNA is critical in terms of calculating Free Energy -> if we can figure out minimum free energy, this gives us good insight on the RNA-protein interactions with respect to how the ribosome is reading the mRNA molecule, whether the free-energy conformation is effective for max translation, and other interaction factors with enzymes and such
    - free energy primarily plays a role in the ability for the mRNA molecule to be detected [cognate = shared structure/affinity vs. noncognate]
        - if the mRNA molecule is non-cognate, then more errors in recognition and thus accuracy of translation and its effectiveness is poor [let ```c``` be cognate frequency, ```nc``` be non cognate frequency]
- Therefore, (∆∆𝐺𝑏𝑖𝑛𝑑 = ∆𝐺𝑏𝑖𝑛𝑑𝑐-∆𝐺𝑏𝑖𝑛𝑑𝑛𝑐 ), which is related to the populations or dissociation constants: 
    - ∆∆𝐺𝑏𝑖𝑛𝑑 = −𝑅𝑇 ln (𝑃𝑐/𝑃𝑛𝑐) = −𝑅𝑇𝑙𝑛(𝐾𝑑𝑛𝑐/𝐾𝑑𝑐) 
- we define excess free energy difference between two states as ∆𝐺 = −𝛽^−1𝑙𝑛 (𝑍𝐵/𝑍𝐴) = −𝛽^−1𝑙𝑛〈exp(−𝛽∆𝑈)〉A

- To simply, we can look into laws of thermodynamics as a TLDR;
    - why does energy flow in certain directions in ways? -> Gibbs Free Energy
    - A system under constant pressure -> change in enthalpy = Change in heat
        - A + B = C, change in enthalpy for C needs to have more energy
        - (\Delta G=\Delta H-T \Delta S) -> Gibbs Free Energy = Change in Enthalpy of the Reaction - Temperature * Change in Entropy
                - Change in enthalpy = heat of reaction 
                - Change in entropy = heat-transfer to melt ice * temperature
- Refer to index for calculatinos of observed free energy with respect to individual types of secondary structures -> https://rna.urmc.rochester.edu/NNDB/turner04/index.html

- ViennaRNA also uses Nearest Neighbour Method: 
    - we consider Gibbes Free Energy (G) as all available energy -> goal is to find mRNA structure whose hydorgen bonds energy requirements are below or equal to Gibbs Free Energy
    - G is function of total energy of a system [H] and entropy [S] and temperature of system [T]

            Enthalpy
            - what happens when system is in movement? a lot of the potential energy is turned into kinetic energy
                - therefore, Total energy [H] or enthalpy have decreased since there is less potential energy after each second in the reaction
            -  in the case of biology, potential energy is the hydrogen bonds or base-pairing bonds between the different nucleotide base pairs

            Entropy
            - when we increase disorder, typically referring to spontaneous reactions -> the random movement of all particles with respect to particles

            Temperature
            - if you have increased temperature, you're ultimately increasing entropy due to movement of molecules
    
    - Therefore, G = H - TS -> therefore, a greater temperature/entropy would reduce Gibbs Free Energy, and a greater enthalpy increases Gibbs Free Energy
        - If there is less Gibbs Energy, it indicates that we have more entropy and less enthalpy, indicating that a spontaneous reaction has just occured as more kinetic and less potential energy [or conversion of potential to kinetic] or <strong> Exergonic </strong>
        - If there is more Gibbs Free Energy, temperature/entropy has decreased to reduce chaos and thus not start a spontaneous reactions ]backwards essentially] or <strong> Exergonic </strong>
        - If Gibbs Free Energy = 0, then there is equilibrium between entropy and temperature along with enthalpy or <strong> Equilibrium </strong>
    
    - ie. in cellular respiration, we break bonds in glucose to release energy, this increases our kinetic energy and thus reduces Gibbs Free Energy, therefore exogenous reaction or releasing heat/energy
    - ie. in photosynthesis, you are assembling energy through CO2 and water in the form of glucose or starch, therefore it is endogenous where Gibbs Free Energy is higher
        * measured in kcal/mol
    - on a graph of free energy to reactino progress, we want the bottom-most corner where during a reaction progress for mRNA folding, we want the free energy to be at its minimal

    - this makes logical sense: loss in energy of system = releasing that energy/kinetic | gain in energy of system  = obtaining that energy/potential
    
    
### Minimum Free Energy Calculation
- for MFE, trying to find the most stable conformation that achieves thermodynamic equilibirum [G=0]
- ViennaRNA uses Zuker's algorithm vs Nussinov's Algorithm since Nussinov has an ineffective method for base pair maximization
- going back to Gibbs Free Energy, G = H-TS where H is potential to perform work and S is measure of disorder
    - goal of problem is to compute change of free energy due to folding from P[unfolded] into P[folded]
- Zuker's algorithm sums up the free energy contributions from single structural elements
    - loops, stacks, etc. are experimentally derived from Turner Rules -> all of these bonds depend on temperature at relative 37deg

<img src="/Users/devpatelio/Downloads/poopfeagaega.png">

- overall Gibbs Free Energy is -4.6kcal/mol, indicating that the system has lost energy, and thus is spontaneous where the T*S [entropy] is greater 
- we can also define secondary structures as algebraic statement and can be solved for Minimum Free Energy through Zuker's algorithm
- the definitions of each structure is well defined as a mathematic statement
- hairpin loops typically known to have size of 3 and more -> length of i:k:j = 5 where k represents loop nucleotides [3 of which]

- Different Structures have different energy contributions: 
    - one exception is multiloop of indefinite length -> simplify by another notation instead of calculating energy of each individual unpaired base
        - traditional: eMultiloop = (i, j, i1, j1..., ik, jk)
        - eMultiLoop(i, j, k, k') = a + bk + ck'
            - a = energy contribution for closing of loop base pairs
            - k = number of inner base pairs
            - k' = number of unpaired bases within loop
- we can define free energy of RNA given RNA structure P of sequence S as: 
    1. Loop Free Energy: Eij(P) = energy contribution of SecondaryStructureFormation(i,j) [type of loop and relative energy contribution]
    2. Total Free Energy: E(P) = ∑Eij(P) -> sum of all total energy contributions of each individual secondary structure formation
- this is all assuming sequence is fixed and no changes are made -> *this is where my project idea would come in
- the problem with this simplified approach, is that we cannot account for crossing RNA structures P of S, where we need to minimize crossing of secondary structure loops 

#### Zuker's Algorithm
- another form of dynamic programming similar to Nussinov's where we traceback
- for RNA sequence S, define Zuker matrix W as matrix of entries Wij for 1 ≤ i ≤ j ≤ n as the following:
    - Wij := min{E(P) | P non crossing RNA ij substructure of S}
        - try to minimize energy summs of each closed loop structure

```
for j - 1 ≤ m:
    Wij = 0

    for i < j - m:
        Wij = min{
            Wij-1
            min@i≤k<j-m * Wi,k-1 + Wk+1,j-1 + E(_any P structure obtained_)
        }

```
- we define ```Wk+1*j-1 + E(_any P structure obtained_)``` as a V-matrix
- for RNA sequence S, define Zuker-matrix V as matrix of entries VIj for 1 ≤ i ≤ j ≤ n as the following: 
    - Vij := min{E(P) | 
        P non-crossing RNA ij substructure of S,
        where (i,j) belongs to a substructure
    }

```
for j-1 ≤ m:
    Vij = np.inf

    for i < j-m:
        Vij = min{
            eH(i,j) #hairpinloop
            Vi+1,j-1 + eS(i,j) #stackingloop
            mini<i'<j'<jVi'j' + eL(i,j,i',j') #interiorloop/bulge
            mink,i<k<j of eM(i, k, j) + ∑1≤k'≤k Vik',jk' #multiloop

        }

```

- W-matrix is the minimum energy sum of all non-cross loops and structures in the sequence [basically the overall structure, defined as i,j-1]
- V-matrix is the sum of all next possible non-crossing structure in the sequence as you move to the next inward sequence [refer to Nussinov's explanation in MIT Section]


- Example: [Slides 16 and onward](https://math.mit.edu/classes/18.417/Slides/rna-prediction-zuker.pdf)


- A simplified explanationL 
    - Given 1' primary RNA sequence, how to compute optimal 2' structure by evaluating overall stability through Minimum Free Energy Calculation and Conformation
    - 2 types of energy: 1) energy given out after reaction of binding [loss of energy, therefore negative] 2) non-binding base pairs [destabilizing energy so increase in energy, therefore positive]
    - greater the negative value, better stability and minimum 
- calculate stacking energy based on the base pairs [as shown on table through turner laws]
- next, you find destabilizing energies of the types of loops given their size and format, which can be derived through the turner laws once again
 

### MIT Computational Biology Review
- simple representation of secondary structure is the ability for base pairs to bond to each other using Hydrogen bonds at different areas
- represented in parentheses notation -> ...((....)).....((.......))...
    - dot is unpaired base, parenthesis is paired base -> each left parenthesis is paired to a right parenthesis, all dots in between parenthesis are loops
- in some instances, secondary structure is helpful to identify anticodon or free-pair sites so that you can understand which parts of the mRNA molecule are effectively going to bind to your ribosome/are the most conserved

#### Finding structure from covariation [referencing distance]:
- looking at homologous sequences to extract secondary structure from mRNA
    seq1 = ACGAAAGU
    seq2 = UAGUAAUA
    seq3 = AGGUGACU
    seq4 = CGGCAAUG
    seq5 = GUGGGAAC
- all sequences have same function, are homologous in nature, and this is based in alignment pattern alone
- in first column, AUACG is complementary to either column UAUGC, and the same applies to second column and seventh column -> this indicates that you create a stem loop between all homologous sequences where you are conserving functionality while changing base pairs
- to conduct multiple alignment, use something called Mutual information statistics
    - M(ij) = Ef(ij)log(f(ij)/f(i)f(j))
- i and j are individual columns of the sequence aligned on a matrix
    - f(i) is frequency of a nucleotide in column i, same goes for j
    - f(i,j) is frequency of a sequence i, j and it is a fraction of total occurences in the matrix over all possible occurences
        - i.e if A-C occurs twice out of 7 rows, then f(ij)AC = 2/7
- this formula is just relative entropy of co-occurence of f(ij) vs independent occurences f(i)(fj)
    - your Null hypothesis of the alignment matrix is really just the independent occurence, but computationally we're trying to look for potential co-occurences
- let's say null hypothesis is true, so for the equation, we simplify the mutual information to: 
    - M(ij) = Ef(ij) * log(1)
    - M(ij) = 0 -> therefore, when all homologous sequences are independent, there is 0 mutual information
        - as a result, we're calculating that co-occurence frequency f(ij) and using that as our metric of mutual information
- mutual information = chi-squared test essentially
- ie. distance calculation is really a product of mutual information and how much do we have -> 0 distance means 0 mutual information, and vice versa
- across different columns, we calculate overall mutual information with respect to covariance, positioning, etc.
- to accurately infer RNA secondary structure by covariance, you need high homology/higher limit to maintain conserved columns to then infer the structure
    - there has to also be sufficient divergence between certain homologous sequences, but need to still be similar to some extent
- sufficient number of sequences is required such that your alignment doesn't get congested

- Energy Minimization Approach -> ▵G_folding = G_unfolded - G folded
                               -> assumption made is that you will occupy minimum free energy state
                               -> ▵G = ▵H - T▵S [enthalpy refers to folding, entropy refers to unfolding]
- these are all based on hydrogen bonds -> high temperatures, pH, and reactivity are all variables that affect level of bonding
- Nussinov's algorithm: focuses on Base Pair maximization and focuses only on enthalpy and NOT entropy
- Scoring system for any base pair, 0 for anything else
    - the key objective was to maximize all total number of base pairs
    - using recursive maximization, you can maximize your score for all possible base pairs
- ACAGT -> largest number of base pairs is 2 since we do not allow crossing -> forms RNA loop with 1 base [not realistic as minimum nucleotides required is 3 for loop structure]
    * when we're looking at the probabilistic function of pairing, each base has a probability of pairing with 1/4 of all base pairs, in RNA this might be different due to Watson-Krigg not necessarily applyinf [and thus you would need to code for that as a cost function], so we can evaluate loops and structures by looking at this probability paired with proximity and free energy
- the problem with base pairing is that the first sequence can base pair with any other sequence, but then cannot be paired back -> there is no logical order to approach the problem
- another idea is to start in middle of sequence and work outwards in both directions
    - write sequence on a matrix, initialize diagonal as 0, and then start from the middle of the matrix and build outward
- Given RNA sequence of length N, define S(i,j) to be the score of the best structure for subsequence at position (i, j)
- 4 possible ways to score optimal structure on (i, j) in terms of actual and nested structures as defined in Nussinov's algorithm
    1. i, j paired together -> if the base pair outward of an already confirmed optimal structure can base pair with the other outward base pair, we set this new outer base pairing as the new confirmed optimal structure, and then provide score
    2. i unpaired
    3. j unpaired
    4. bifurcation [junction/middle base pair connecting i and j] -> if your two outward base pairs bind to individual bases in the internal already confirmed structure and not to each other, so your parsing is a product of the outside

- Example: ACAGTTACAGTA
*Status Quo*
    - we know that if A and the first T pair together, and then C pairs with G, then two possible pairings
    - moving on, the next T binds to A, A binds to T, and C binds to G
*Nussinov*
    - now you're working from the inside, so you find the base pairing that is closest to middle T in sequence, which is A
    - as you work outwards, the scope of your parsed sequence [square brackets] also increases dramatically
    - as you work out, C cannot base pair to anything so then you add that to the sequence, and the same goes for the last ending As
        - * for the last A base pairs however, notice that we can refer to bifurcation where now, these bases can pair with the optimal structure inside closes to it

- Dynamic programming is as follows: 
    - S(i, j) = score of the optimal structure of the subsequence made by (i, j)
    - Initialize an NxN matric S with S(i, i) = S(i, i-1) = 0 [basically make the diagonal a 0 across middle AND subdiagonal of 0 [subdiagonal along the middle diagonal to the end] 
    - Trace back from S (1, N) [top right] to diagonal to determine optimal structure by maximizing score we pre-established for 4 possible states


### Comparing and Implementing Multiple Approaches
- in RNA, you form watson crick pairs [AU, GC] and also wobble pairs [GU]
- RNA conserves secondary structure more than their primary structure, hence you can make base pair substitutions or mutations and it wno't affect structure, function, or genetic information in few cases

*Nussinov's Algorithm*
- starts from middle of sequence, and assigns function S(i,j)
    - S(i,j) = 1 if i,j are complementary, otherwise it is 0
- then recursively calculates the leements of a score matrix M(i,j) which is the maximal number of base pairs that can be found in subsequence i...j
    *Initialization*:
    - M(i, i-1) = 0 for i = 2 to L -> entire i series initialized to 0
    - M(i, i) = 0 for i = 1 to L

    *Recursion*: Insert Image
- final score is M(1, L) where L is length of sequence -> O(L^3) time complexity
- in recursion, you push new values for (i,j) until you get the optimal score -> maximized number of base pairs is simplistic criteria and does not yield accurate results

*Zuker Algorithm*
- divides secondary structure into elements that can be described as graphs, sometimes called loops
    - assigns free energy based on the face of those graphs -> structure with lowest free energy is optimal structure
- has 2 main matrices: W(i,j) and V(i,j)
    1. W(i,j) is total free energy of subsequence i to j
    2. V(i, j) is total free energy of subsequence i to j IF they pair, else V(i,j) set as infinity

```
V(i,j) = min(
    FH(i,j)
    min[FL(i, j, h, k) + V(h, k)] where i < h < k < j
    min[W(i+1, k) + W(k+1,j-1)] where i + 1 < k < j-1
)
```
- all you see here is the different scores for types of loops
- FH(i,j) calculates energy of a hairpin loop between nucleotides i, j
- FL(i, j, h, k) is energy of secondary loop [just vaguely defined] including stack, bluge, and interior loop
    - i and j indicate the nucleotides that are a part of the main chain/start of the loop, h, and k indicate the actual nucleotides part of the main chain connected to i and j respectively
        - if you notice, that's why we're calcualting the free energy of V(h, k) only and disincluding i, j but using it in the recursive step
- W(i+1, k) + W(k+1,j-1) is the bifurcation loop -> think of a bifurcation as the junction of two individual loops by 2 nucleotides (k & k+1)
    - first loop is i+1 -> k [i is considered a part of the initial starting but not the loop, similar to secondary loops] 
    - second loop is k+1 to j-1 -> note that k->k+1 is paired base
- reason we don't include i and j in the secondary loops apart from the hairpin and additionally the bifurcation is that the i and j base pairs are already base-paired as a part of the stem

![](https://slidetodoc.com/presentation_image/13f7a8af6b4c62c7493b2b71ed41b4b9/image-17.jpg)
![](https://www.researchgate.net/profile/Shahid-Mahboob/publication/258703336/figure/fig2/AS:341269298597896@1458376349658/Nussinov-algorithm-steps-a-the-four-cases-examined-by-the-dynamic-programming.png)
![](http://bioinfopakistan.ucoz.com/_nw/1/16213284.jpg)
![](http://bioinfopakistan.ucoz.com/_nw/1/55469176.jpg)
![](https://bayesianneuron.com/wp-content/uploads/2019/02/subsequence.png)
![](https://ars.els-cdn.com/content/image/3-s2.0-B9780128012130000137-f13-12-9780128012130.jpg)

- in reference to the bifurcation loop, how do we recurse through that independently?
    - bifurcation is the whole energy across entire stem-loop structure, so it can additionally be used for the entire mRNA molecule
```
W(i, j) = min(
    W(i+1, j)
    W(i, j-1)
    V(i,j)
    min[W(i+k), W(k+1, j) where i < k < j]
)
```

- when you go from i+1 to j, you're calculating the total free energy for the entire structure disincluding the final j base pair
- for V(i, j), you can calculate the minimum free energy as an entire hairpin loop
- the final W(i+k), W(k+1, j) is the actual bifurcation loop that goes from one base pair -> one loop, and then the next loop -> next base pair
- if you compare SS between Nussinov and Zuker, you can see that Zuker has less base pairs while Nussinov has more base pairs [which makes sense given role of both algorithms]
- in some cases, Zuker's algorithm uses compelx energy functions which are not often accurate enough -> minimum free energy "optimal structure" cannot be considered btter than structures with very close free energy
    - hence, we use suboptimal structures as well

#### Stochastic Free Grammar
- context-free grammaer {c, g, a, u} and some non-terminals 
    - to calculate best structure based on Minimum Free Energy, we need to introduce probabilities for each grammar rule, hence *Stochastic Context-Free Grammar* is suitable
- Structure's probability = Joint probability of all grammar rules used to derive the structure and summed over all possible grammar derivations (parse trees) [all branches in tree]
    - predicted strucutre is the one with the highest structure probability

S -> aS | Sa | aSa | SS | e where a-{A, U, C, G}
- Structural probability = S
    - each aS, Sa, aSa, SS is calculating the probability of that specific bonding [unpaired a and Structure, Structure and unpaired a, 2as paired with on structure, no a]
- for each kind of structure/loop S, you can calculate the log probability of the nonterminal grammar rule to the structure of the loop
    - ie. λ(i+1, j) + log P(S -> aS) #a is base pair, S is non-terminal grammar = the loop type [any other secondary loop/bulge]
          λ(i, j-1) + log P(S -> Sa) #same synatx [any other secondary loop/bulge]
          λ(i+1, j-1) + log P(S -> aSa) #grammar rule for just the loop, not including base paired i, j [hairpin in this instance]
          max[λ(i,k) + λ(k+1,j) + log P(S -> SS)] #bifurcation loop grammar structure
- ∑ P = 1 where all probabilities calculated above should equal 1
    - therefore, given a sequence of nucleotides from i -> j, calculate probabilities of all possible kinds of loop formations 
- we calculate this probability with a co-variance matrix for probability of base pairing for any 2 nucleotides
- why is it called context-free? well, the rules we derive in our grammar betw. nonterminal and terminal
    - Context-free grammar G defined as: 
        ```
        G = (V, ∑, R, S)
        1. each element v 𝝐 V is nonterminal [represents different type of phrase/clause in sentence]
        2. ∑ is finite ste of temrinals, disjoint from V, make up the possible letters, words in a dictionary (i.e. A, G, C, U)
        3. R is finite relation in V x (V U X)* or the jacobian
        4. S is start element of setence, it is often nonterminal and part of V
        ```

    - for RNA, ``` G = ({S}, {a}, P, S) ```
        - S is all non-terminal variables
        - a is our grammar, in this case base pairs A, C, U, G
        - P is the finite relation [also known as production rule] where for element in V, its relation to elements in ∑ is characterized
            - ie. if given (a,b) 𝝐 R, (a) 𝝐 V, (b) 𝝐 (V U E)*
                - we use the latter to draw relationships, so  ``` a -> b1, a -> b2, a -> b1|b2``` -> these kinds of relationships are the jacobian or production rule defined as R

        - in the case of RNA, P represents all possible productions as the following: 
            ``` S -> aSa, S -> aS, S -> Sa, S -> SS, S -> e ##e is denoted as empty string, so non-terminal includes spaces
            ```
- Ultimate goal: using P production rules and input of a start symbol/string to a final value
- ![refer to derivation and syntax trees](https://en.wikipedia.org/wiki/Context-free_grammar)
     - to get to final string, you start with start symbol and through a sequence of steps, you can recreate the string
- you can format these grammar rules into trees/parses -> REFER TO AUTOMOTONS
- codon optimization is status quo → converting mRNA codon space problem into lattice parsing reduces design space
    1. formulate mRNA design space as a deterministic finite-state automaton (DFA) = word lattice → explanation above provides more insight on how multiple states and transitions encoded as codon switches + start and stop codons AUG and UAG are the accepting state
    2. generalize single sequence by lattice parsing that folds all sequences in the finite-state automaton indirectly → look into CFG-DFA
        - CFG, or context-free grammar
            - grammars: several rules that describe relationships between an input and output, so given any input, you can automatically figure out what the substitution can be
                - ie. A → B, Codon A → Codon C or D or Z [substitutions based on grammar rules]
            - also have regular grammars where rules are in particular form [higher specificity]
                - ie. variable makes variable, variable makes string, variable to terminal, etc. → you can use this to make regular grammars on mRNA
    3. CFG represents the folding energy model 
    4. mRNA design problem formulated by paper by **protein sequence = p0....p[n]** where **pi** is an amino acid
        1. goal is to search among all possible mRNA sequences [**r[** that translate into protein **p**, then select the best **mRNA sequence** [**r*(p)]** defined as sequence based on fitness model of minimum folding free energy change
        2. *mRNA(*p) = {r | *protein(*r) = p} represents set of candidate mRNA sequences, *structures(*r) is set of all possible secondary structure for mRNA sequence **r**, and *▵G(r, s)* is freen energy change of structure s for mRNA r according to basic energy model
            1. hard-routed algorithm *which is conventional knowledge* is to fold each mRNA individually, log its MFE, and then find best MFE
- **DFA Representations [**Creating these finite-state automota]
    - draw out basic graph embedding of the possible different mRNA pathways
    - learn grammar rules based on score determined by fitness model on the sequence of the model → not sure about this approach, need to learn more but ultimately its about maximizing learned rules based on highest score [0 is always bad, but not necessarily terrible since it validates current design norms and provides more insight]
    - convert best optimal string to DFA where each index becomes a DFA starte, and span on string becomes path between two states
    - Lattice parsing for DFA in A summarized the approach of the DFA where it calculates score of grammar rules on given regions of the rules and finds best ones → DFA is basically a directed graph with start and stop nodes + branching at a node denotes possibility for ambiguity in protein structure
- so we can parse these automotas ultimately in a recurvise nature

- in the case of Zuker's algorithm, we define the rules of our stochastic-context free grammar as probabilistic representations of free energy and type of secondary structure formed
- then, the sum of the probabilities = 1, therefore we need to parse our automoton given the SS formed to get to a final representation [reverse parsing] but defined only by the condition that MFE has to be the lowest
- what Zuker's does different than Nussinov's is that it defined a co-variation matrix to record the probability of base-pairing for any two nucleotides



### SS Folding Grammar
### SS Landscape
### Partiion Function + Equilibirum Probability
### Suboptimal Conformations
### RNA-RNA Interactions
### Locally Stable SS
### Comparitive Structure Prediction
### Distance Measures

In [1]:
import os 
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import operator as op
from functools import reduce

In [5]:
rna = 'AAUACUCCGUUGCAGCAU'

class MinimumFreeEnergy():
    def __init__ (self, seq):
        super(MinimumFreeEnergy, self).__init__()
        self.seq = seq
        N = len(self.seq)
        W = np.zeros((N,N))
        V = W.copy()

        for i in range(N):
            for j in range(N):
                if j-i < 4: #minimum size of loop, initalize as np.inf
                        
                    W[i][j] = np.inf
                    V[i][j] = np.inf
                
                else:
                    V[i][j] = np.nan
                    W[i][j] = np.nan

    def turner_energy(self):
        #i_1,j_1,i_2,j_2
        b = ['A', 'C', 'G', 'U']
        all_bps = []

        for i in b:
            for j in b:
                if i=='A' and j=='U' or i=='U' and j=='A' or i=='G' and j == 'C' or i=='C' and j=='G' or i =='G' and j == 'U' or i=='U' and j=='G':
                    all_bps.append(i+j)
                else:
                    pass

        AU_tt = np.array([-0.9,-1.8,-2.3,-1.1,-0.5,-0.7])
        CG_tt = np.array([-2.1,-2.9,-3.4,-2.3,-1.5,-1.5])
        GC_tt = np.array([-1.7,-2,-2.9,-1.8,-1.3,-1.5])
        UA_tt = np.array([-0.9,-1.7,-2.1,-0.9,-0.7,-0.5])
        GU_tt = np.array([-0.9,-1.7,-2.1,-0.9,-0.5,-0.5])
        UG_tt = np.array([-0.9,-1.7,-2.1,-0.9,0.6,-0.5])
        
        te_m = np.array((AU_tt,CG_tt,GC_tt,UA_tt,GU_tt,UG_tt))


        internal_tt = [0,0.8,1.3,1.7,2.1,2.5,2.6,2.8]
        bulge_tt = [3.3,5.2,6,6.7,7.4,8.2,9.1,10]
        hairpin_tt = [0,0,7.4,5.9,4.4,4.3,4.1,4.1]
        
        ss_m = np.array((internal_tt, bulge_tt, hairpin_tt))
        


        ss_rules = dict(zip(['internal', 'bulge', 'hairpin'], ss_m))
        te_rules = dict(zip(all_bps, te_m))
    
        return all_bps, te_rules, te_m, ss_m, ss_rules
        
        
    def hairpin(self, i, j):
        all_bps, _, _, ss_m, ss_rules = MinimumFreeEnergy(self.seq).turner_energy()

        hairpin_seq = self.seq[i+1:j]
        assert [k in all_bps for k in [hairpin_seq[x]+hairpin_seq[len((hairpin_seq)-x)] for x in range(len(hairpin_seq)-1)]]

        eH = ss_rules['hairpin'][len(hairpin_seq)]
        return eH
        

    def stacking(self, i, j):
        all_bps, te_rules, te_m, _, _ = MinimumFreeEnergy(self.seq).turner_energy()

        pair_bottom = [self.seq[i], self.seq[j]]
        pair_top = [self.seq[i+1], self.seq[j-1]]
        assert self.seq[i]+self.seq[j] and self.seq[i+1]+self.seq[j-1] in all_bps

        eS = te_m[''.join(pair_bottom)][all_bps.index(''.join(pair_top))]
        return eS


    def internal(self, i, j, i_p, j_p):
        all_bps, _, _, ss_m, ss_rules = MinimumFreeEnergy(self.seq).turner_energy
        
        


        return None
    


    def multiloop():
        pass

        

MinimumFreeEnergy.turner_energy(rna)